# One vs Rest

In [1]:
import pylab as pl
import numpy as np
import pandas as pd  
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.linear_model import LogisticRegression

In [2]:
#read the data set
df = pd.read_csv('./data/joinedData.csv', sep=r'\s*(?:\||\#|\,)\s*',
                 engine='python')

In [3]:
from sklearn.feature_selection import RFE

#change the 5 tumour types to numbers
Class = {'LUAD': 0,'BRCA': 1,'KIRC': 2,'PRAD': 3,'COAD': 4} 
#this is where we add the class to the table
df.Class = [Class[item] for item in df.Class]
#drop the 2 unnamed table because we do not need them
df = df.drop('Unnamed: 0',1)
df = df.drop('Unnamed: 0.1',1)
df

,Class,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
0,3,0.000000,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.000000,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.000000
1,0,0.000000,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.000000,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.000000
2,3,0.000000,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.000000,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.000000
3,3,0.000000,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.000000,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.000000
4,1,0.000000,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.000000,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.000000
5,3,0.000000,3.467853,3.581918,6.620243,9.706829,0.0,7.758510,0.000000,0.000000,...,5.726657,8.602588,9.928339,6.096154,9.816001,11.556995,9.244150,9.836473,5.355133,0.000000
6,2,0.000000,1.224966,1.691177,6.572007,9.640511,0.0,6.754888,0.531868,0.000000,...,5.105904,7.927968,9.673966,1.877744,9.802692,13.256060,9.664486,9.244219,8.330912,0.000000
7,3,0.000000,2.854853,1.750478,7.226720,9.758691,0.0,5.952103,0.000000,0.000000,...,5.297833,8.277092,9.599230,5.244290,9.994339,12.670377,9.987733,9.216872,6.551490,0.000000
8,1,0.000000,3.992125,2.772730,6.546692,10.488252,0.0,7.690222,0.352307,0.000000,...,6.721974,9.597533,9.763753,7.933278,10.952880,12.498919,10.389954,10.390255,7.828321,0.000000
9,3,0.000000,3.642494,4.423558,6.849511,9.464466,0.0,7.947216,0.724214,0.000000,...,6.020051,8.712809,10.259096,6.131583,9.923582,11.144295,9.244851,9.484299,4.759151,0.000000


In [4]:
X = df.drop('Class', axis=1).values
y = df['Class'].values
y = np.asarray(y)

In [5]:
# feature extraction
#model = LogisticRegression()
#rfe = RFE(model, 3)
#fit = rfe.fit(X, y)
#print("Num Features: %d") % fit.n_features_
#print("Selected Features: %s") % fit.support_
#print("Feature Ranking: %s") % fit.ranking_

In [6]:
#Standardize data
X = (X - X.mean()) / (X.max() - X.min())

#Split the training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)  
X_train

array([[-0.31009073, -0.17314025, -0.17092733, ...,  0.14479179,
        -0.16756171, -0.31009073],
       [-0.31009073, -0.24838072, -0.1657764 , ...,  0.15107929,
        -0.02477067, -0.31009073],
       [-0.31009073, -0.08768639, -0.12152632, ...,  0.15792214,
         0.0799575 , -0.31009073],
       ...,
       [-0.31009073, -0.20680766, -0.14623389, ...,  0.14587579,
        -0.03078254, -0.31009073],
       [-0.31009073, -0.11563346, -0.12269672, ...,  0.15631198,
        -0.14153606, -0.29298897],
       [-0.31009073, -0.19012855, -0.15522276, ...,  0.15125456,
        -0.09566869, -0.31009073]])

In [7]:
##Predict
df2 = df.drop('Class',1)
test = df2.iloc[[800]]
test

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
800,0.0,2.325242,3.805932,6.530246,9.560367,0.0,7.957027,0.0,0.0,0.0,...,6.403075,8.594354,10.243079,9.139459,10.102934,11.641081,10.607358,9.844794,4.550716,0.0


In [8]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import accuracy_score, classification_report

In [9]:
##Logistic Regression
clf1 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr').fit(X_train, y_train)
print("Logistic Regression prediction :",clf1.predict(X_test))
print("Logistic Regression probability :",clf1.predict_proba(test))
lrTest = clf1.predict(X_test)
print("Logistic Regression score :", accuracy_score(y_test,lrTest))

Logistic Regression prediction : [2 2 1 1 2 1 0 0 2 1 2 2 1 1 3 3 1 3 1 1 2 4 1 3 1 2 1 3 1 2 0 0 1 0 0 0 4
 0 1 4 2 2 1 0 0 1 2 0 1 1 0 3 1 1 4 0 1 2 2 1 0 0 1 1 2 3 3 0 2 4 1 1 1 0
 3 3 3 1 1 3 3 2 0 3 2 1 1 1 1 3 2 1 4 1 4 1 1 1 3 1 1 2 1 3 3 3 2 1 1 2 0
 2 1 4 0 0 1 1 2 1 3 3 1 1 4 3 2 2 3 1 1 1 1 1 2 1 0 2 3 4 0 4 4 2 3 3 0 1
 0 0 0 0 2 3 0 0 1 1 0 1 4]
Logistic Regression probability : [[4.99999997e-01 2.93178543e-41 1.33880218e-25 4.99999997e-01
  6.98983658e-09]]
Logistic Regression score : 0.9937888198757764


In [10]:
##Logistic Regression CV
clf2 = LogisticRegressionCV(cv=5, random_state=0, multi_class='ovr').fit(X_train, y_train)
print("Logistic Regression CV prediction :",clf2.predict(X_test))
print("Logistic Regression CV probability :",clf2.predict_proba(test))
lrTest = clf2.predict(X_test)
print("Logistic Regression score CV :", accuracy_score(y_test,lrTest))

Logistic Regression CV prediction : [2 2 1 1 2 1 0 0 2 1 2 2 1 1 3 3 1 3 1 1 2 4 1 3 1 2 1 3 1 2 0 0 1 0 0 0 4
 0 1 4 2 2 1 0 0 1 2 0 1 1 0 3 1 1 4 0 1 2 2 1 0 0 1 1 2 3 3 0 2 4 1 1 1 0
 3 3 3 1 1 3 3 2 0 3 2 1 1 1 1 3 2 1 4 1 4 1 1 1 3 1 1 2 1 3 3 3 2 1 1 2 0
 2 1 4 0 0 1 1 2 1 3 3 1 1 4 3 2 2 3 1 1 1 1 1 2 1 0 2 3 4 0 4 4 2 3 3 0 1
 0 0 0 0 2 3 0 0 1 1 0 1 4]
Logistic Regression CV probability : [[4.99999999e-01 1.40534199e-13 1.09026100e-18 4.99999999e-01
  2.90299163e-09]]
Logistic Regression score CV : 0.9937888198757764


In [11]:
##Gaussian Process Classifier
kernel = 1.0 * RBF(1.0)
clf3 = GaussianProcessClassifier(multi_class = 'one_vs_rest',kernel=kernel,random_state=0).fit(X_train, y_train)
print("Gaussian Process Classifier prediction :",clf3.predict(X_test))
print("Gaussian Process Classifier probability :",clf3.predict_proba(test))
lrTest = clf3.predict(X_test)
print("Gaussian Process Classifier score :", accuracy_score(y_test,lrTest))

Gaussian Process Classifier prediction : [2 4 1 1 2 1 4 4 2 1 2 2 1 4 4 3 4 3 4 4 2 4 4 3 1 2 4 3 1 4 0 4 1 4 0 4 4
 4 1 4 2 2 4 4 4 4 2 4 4 1 0 3 4 4 4 4 1 2 2 4 4 4 4 4 2 3 4 4 4 4 4 1 1 4
 3 3 3 1 4 3 3 2 4 3 2 4 1 1 1 4 4 1 4 4 4 1 4 1 3 1 4 2 4 3 3 3 2 1 4 2 4
 2 1 4 0 4 1 4 2 4 3 3 4 1 4 3 2 2 3 4 4 4 4 1 2 4 0 2 3 4 0 4 4 2 3 3 0 4
 0 4 4 4 2 3 0 0 1 1 0 1 4]
Gaussian Process Classifier probability : [[0.2 0.2 0.2 0.2 0.2]]
Gaussian Process Classifier score : 0.639751552795031


In [12]:
##Linear SVC
kernel = 1.0 * RBF(1.0)
clf4 = LinearSVC(multi_class='ovr',random_state=0, tol=1e-5).fit(X_train, y_train)
print("Linear SVC prediction :",clf4.predict(X_test))
lrTest = clf4.predict(X_test)
print("Linear SVC score :", accuracy_score(y_test,lrTest))

Linear SVC prediction : [2 2 1 1 2 1 0 0 2 1 2 2 1 1 3 3 1 3 1 1 2 4 1 3 1 2 1 3 1 2 0 0 1 0 0 0 4
 0 1 4 2 2 1 0 0 1 2 0 1 1 0 3 1 1 4 0 1 2 2 1 0 0 1 1 2 3 3 0 2 4 1 1 1 0
 3 3 3 1 1 3 3 2 0 3 2 1 1 1 1 3 2 1 4 0 4 1 1 1 3 1 1 2 1 3 3 3 2 1 1 2 0
 2 1 4 0 0 1 1 2 1 3 3 1 1 4 3 2 2 3 1 1 1 1 1 2 1 0 2 3 4 0 4 4 2 3 3 0 1
 0 0 0 0 2 3 0 0 1 1 0 1 4]
Linear SVC score : 1.0


In [13]:
from sklearn.multiclass import OneVsRestClassifier

#MultiClassOnevsRestClassifier

clf5 = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
print("MultiClassOnevsRestClassifier prediction :",clf5.predict(X_test))
lrTest = clf5.predict(X_test)
print("MultiClassOnevsRestClassifier score :", accuracy_score(y_test,lrTest)) 

MultiClassOnevsRestClassifier prediction : [2 2 1 1 2 1 0 0 2 1 2 2 1 1 3 3 1 3 1 1 2 4 1 3 1 2 1 3 1 2 0 0 1 0 0 0 4
 0 1 4 2 2 1 0 0 1 2 0 1 1 0 3 1 1 4 0 1 2 2 1 0 0 1 1 2 3 3 0 2 4 1 1 1 0
 3 3 3 1 1 3 3 2 0 3 2 1 1 1 1 3 2 1 4 0 4 1 1 1 3 1 1 2 1 3 3 3 2 1 1 2 0
 2 1 4 0 0 1 1 2 1 3 3 1 1 4 3 2 2 3 1 1 1 1 1 2 1 0 2 3 4 0 4 4 2 3 3 0 1
 0 0 0 0 2 3 0 0 1 1 0 1 4]
MultiClassOnevsRestClassifier score : 1.0


### References

#### https://scikit-learn.org/stable/modules/multiclass.html
#### https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC
#### https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html#sklearn.gaussian_process.GaussianProcessClassifier
#### https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
#### https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV

###  References in Vancouver
#### 1. 1.12. Multiclass and multilabel algorithms — scikit-learn 0.20.1 documentation [Internet]. Scikit-learn.org. [cited 4 December 2018]. Available from: https://scikit-learn.org/stable/modules/multiclass.html
#### 2. sklearn.svm.LinearSVC — scikit-learn 0.20.1 documentation [Internet]. Scikit-learn.org. [cited 4 December 2018]. Available from: https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC
#### 3. sklearn.gaussian_process.GaussianProcessClassifier — scikit-learn 0.20.1 documentation [Internet]. Scikit-learn.org. [cited 4 December 2018]. Available from: https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html#sklearn.gaussian_process.GaussianProcessClassifier
#### 4. sklearn.linear_model.LogisticRegression — scikit-learn 0.20.1 documentation [Internet]. Scikit-learn.org. [cited 4 December 2018]. Available from: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
#### 5. 3.2.4.1.5. sklearn.linear_model.LogisticRegressionCV — scikit-learn 0.20.1 documentation [Internet]. Scikit-learn.org. [cited 4 December 2018]. Available from: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html#sklearn.linear_model.LogisticRegressionCV

In [ ]:
##Logistic Regression
clf1 = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr')
rfe = rfe = RFE(clf1, 3)
rfe.fit(X_train, y_train)
#print("Logistic Regression prediction :",clf1.predict(X_test))
#print("Logistic Regression probability :",clf1.predict_proba(test))
#lrTest = clf1.predict(X_test)
#print("Logistic Regression score :", accuracy_score(y_test,lrTest))
# print summaries for the selection of attributes
print(rfe.support_)
print(rfe.ranking_)